In [1]:
# Import Main Dependencies
import pandas as pd
import json
import requests
from pprint import pprint

# Import credentials string for PostgreSQL database access
from config import credentials

# Library and methods to connect and interact with database
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from sqlalchemy.orm import sessionmaker

In [2]:
# Dataset 1: Read CSV dataset downloaded from City of Chicago Health Services Food Inspections into dataframe
pd.options.display.max_colwidth = 200
past_data_csv_df = pd.read_csv("Resources/Food_Inspections_past.csv")

past_data_csv_df.head(10)

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,67757,DUNKIN DONUTS/BASKIN-ROBBINS,DUNKIN DONUTS/BASKIN-ROBBINS,1380279.0,Restaurant,Risk 2 (Medium),100 W RANDOLPH ST,CHICAGO,IL,60601.0,01/04/2010,Tag Removal,Pass,NaN,41.884586,-87.631010,"(41.88458626715456, -87.63101044588599)"
1,52234,Cafe 608,Cafe 608,2013328.0,Restaurant,Risk 1 (High),608 W BARRY AVE,CHICAGO,IL,60657.0,01/04/2010,License Re-Inspection,Pass,NaN,41.938007,-87.644755,"(41.938006880423615, -87.6447545707008)"
2,70269,mr.daniel's,mr.daniel's,1899292.0,Restaurant,Risk 1 (High),5645 W BELMONT AVE,CHICAGO,IL,60634.0,01/04/2010,License Re-Inspection,Pass,NaN,41.938443,-87.768318,"(41.93844282365204, -87.76831838068422)"
3,67733,WOLCOTT'S,TROQUET,1992040.0,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613.0,01/04/2010,License Re-Inspection,Pass,NaN,41.961606,-87.675967,"(41.961605669949854, -87.67596676683779)"
4,104236,TEMPO CAFE,TEMPO CAFE,80916.0,Restaurant,Risk 1 (High),6 E CHESTNUT ST,CHICAGO,IL,60611.0,01/04/2010,Canvass,Fail,"18. NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS PROTECTED/RODENT PROOFED, A WRITTEN LOG SHALL BE MAINTAINED AVAILABLE TO THE INSPECTORS - Comments: All necessary control measures shall be used ...",41.898431,-87.628009,"(41.89843137207629, -87.6280091630558)"
5,67732,WOLCOTT'S,TROQUET,1992039.0,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613.0,01/04/2010,License Re-Inspection,Pass,NaN,41.961606,-87.675967,"(41.961605669949854, -87.67596676683779)"
6,67738,MICHAEL'S ON MAIN CAFE,MICHAEL'S ON MAIN CAFE,2008948.0,Restaurant,Risk 1 (High),8750 W BRYN WAWR AVE,CHICAGO,IL,60631.0,01/04/2010,License,Fail,"18. NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS PROTECTED/RODENT PROOFED, A WRITTEN LOG SHALL BE MAINTAINED AVAILABLE TO THE INSPECTORS - Comments: must provide pest control log book. | 32. FOO...",NaN,NaN,NaN
7,67760,MC INTOSH'S,MC INTOSH'S,1648623.0,NaN,Risk 1 (High),748 W 61ST ST,CHICAGO,IL,60621.0,01/05/2010,Out of Business,Fail,NaN,41.783503,-87.644386,"(41.783502980089736, -87.64438637632864)"
8,78240,OBAMA FOOD MART,NaN,2004238.0,Grocery Store,Risk 3 (Low),1800 E 79TH ST,CHICAGO,IL,60649.0,01/05/2010,License,Fail,NaN,41.751655,-87.580828,"(41.75165523778611, -87.58082842408824)"
9,74258,MAKIA FOOD,MAKIA FOOD,2009488.0,NaN,Risk 3 (Low),2458 S CALIFORNIA AVE,CHICAGO,IL,60608.0,01/05/2010,License Re-Inspection,Pass,NaN,41.846516,-87.695423,"(41.84651642859939, -87.69542345938576)"


In [3]:
# Drop columns not necessary for this project from dataframe
past_data_csv_df= past_data_csv_df.drop(columns=['Latitude', 'Longitude', 'Location'])
past_data_csv_df.head(20)

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations
0,67757,DUNKIN DONUTS/BASKIN-ROBBINS,DUNKIN DONUTS/BASKIN-ROBBINS,1380279.0,Restaurant,Risk 2 (Medium),100 W RANDOLPH ST,CHICAGO,IL,60601.0,01/04/2010,Tag Removal,Pass,NaN
1,52234,Cafe 608,Cafe 608,2013328.0,Restaurant,Risk 1 (High),608 W BARRY AVE,CHICAGO,IL,60657.0,01/04/2010,License Re-Inspection,Pass,NaN
2,70269,mr.daniel's,mr.daniel's,1899292.0,Restaurant,Risk 1 (High),5645 W BELMONT AVE,CHICAGO,IL,60634.0,01/04/2010,License Re-Inspection,Pass,NaN
3,67733,WOLCOTT'S,TROQUET,1992040.0,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613.0,01/04/2010,License Re-Inspection,Pass,NaN
4,104236,TEMPO CAFE,TEMPO CAFE,80916.0,Restaurant,Risk 1 (High),6 E CHESTNUT ST,CHICAGO,IL,60611.0,01/04/2010,Canvass,Fail,"18. NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS PROTECTED/RODENT PROOFED, A WRITTEN LOG SHALL BE MAINTAINED AVAILABLE TO THE INSPECTORS - Comments: All necessary control measures shall be used ..."
5,67732,WOLCOTT'S,TROQUET,1992039.0,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613.0,01/04/2010,License Re-Inspection,Pass,NaN
6,67738,MICHAEL'S ON MAIN CAFE,MICHAEL'S ON MAIN CAFE,2008948.0,Restaurant,Risk 1 (High),8750 W BRYN WAWR AVE,CHICAGO,IL,60631.0,01/04/2010,License,Fail,"18. NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS PROTECTED/RODENT PROOFED, A WRITTEN LOG SHALL BE MAINTAINED AVAILABLE TO THE INSPECTORS - Comments: must provide pest control log book. | 32. FOO..."
7,67760,MC INTOSH'S,MC INTOSH'S,1648623.0,NaN,Risk 1 (High),748 W 61ST ST,CHICAGO,IL,60621.0,01/05/2010,Out of Business,Fail,NaN
8,78240,OBAMA FOOD MART,NaN,2004238.0,Grocery Store,Risk 3 (Low),1800 E 79TH ST,CHICAGO,IL,60649.0,01/05/2010,License,Fail,NaN
9,74258,MAKIA FOOD,MAKIA FOOD,2009488.0,NaN,Risk 3 (Low),2458 S CALIFORNIA AVE,CHICAGO,IL,60608.0,01/05/2010,License Re-Inspection,Pass,NaN


In [4]:
# Rename columns to get ready for database upload
name_change_df = past_data_csv_df.rename(columns={'Inspection ID':'inspection_id',
'DBA Name' : 'business_name',
'AKA Name' : 'aka_name',
'License #' : 'license_id',
'Facility Type' : 'facility_type',
'Risk' : 'risk',
'Address' : 'address',
'City' : 'city',
'State' : 'state_id',
'Zip': 'zip',
'Inspection Date' : 'inspection_date',
'Inspection Type' : 'inspection_type',
'Results': 'results',
'Violations' : 'violations'})

In [5]:
# Inspect dataframe column data types
past_data_csv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172880 entries, 0 to 172879
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Inspection ID    172880 non-null  int64  
 1   DBA Name         172880 non-null  object 
 2   AKA Name         170663 non-null  object 
 3   License #        172872 non-null  float64
 4   Facility Type    168288 non-null  object 
 5   Risk             172819 non-null  object 
 6   Address          172880 non-null  object 
 7   City             172787 non-null  object 
 8   State            172858 non-null  object 
 9   Zip              172840 non-null  float64
 10  Inspection Date  172880 non-null  object 
 11  Inspection Type  172879 non-null  object 
 12  Results          172880 non-null  object 
 13  Violations       126458 non-null  object 
dtypes: float64(2), int64(1), object(11)
memory usage: 18.5+ MB


In [6]:
# Dataset 2: Get complimentary restaurant and inspection details from API
url = 'https://data.cityofchicago.org/resource/qizy-d2wf.json'
response_2018 = requests.get(url).json()


pprint(json.dumps(response_2018, indent=4, sort_keys=True))

('[\n'
 '    {\n'
 '        ":@computed_region_43wa_7qmu": "18",\n'
 '        ":@computed_region_6mkv_f3dw": "21849",\n'
 '        ":@computed_region_awaf_s7ux": "13",\n'
 '        ":@computed_region_bdys_3d7i": "665",\n'
 '        ":@computed_region_vrxf_vc4k": "6",\n'
 '        "address": "2324 W GIDDINGS ST ",\n'
 '        "aka_name": "PACIUGO CHICAGO",\n'
 '        "city": "CHICAGO",\n'
 '        "dba_name": "PACIUGO CHICAGO",\n'
 '        "facility_type": "Mobile Food Dispenser",\n'
 '        "inspection_date": "2023-01-17T00:00:00.000",\n'
 '        "inspection_id": "2569731",\n'
 '        "inspection_type": "Canvass",\n'
 '        "latitude": "41.96765092713031",\n'
 '        "license_": "2240574",\n'
 '        "location": {\n'
 '            "latitude": "41.96765092713031",\n'
 '            "longitude": "-87.6875610952487",\n'
 '            "needs_recoding": false\n'
 '        },\n'
 '        "longitude": "-87.6875610952487",\n'
 '        "results": "Out of Business",\n'
 '     

In [9]:
# Prepare Column Heading list
clist = ['address', 'aka_name', 'city', 'business_name', 'facility_type', 'inspection_date',
        'inspection_id',  'inspection_type', 'license_id', 'results', 'risk', 'state_id', 'violations', 'zip']
# Empty list to hold all data rows
data_rows = []
# Extract data from response and store in
for x in response_2018:
    dlist = []
    dlist.append(x['address'])
    try:
        dlist.append(x['aka_name'])
    except KeyError:
        dlist.append(' ')
    try:    
        dlist.append(x['city'])
    except KeyError:
        dlist.append(' ')
        
    dlist.append(x['dba_name'])
    try:
        dlist.append(x['facility_type'])
    except KeyError:
        dlist.append(' ')
    dlist.append(x['inspection_date'])
    dlist.append(x['inspection_id'])
    dlist.append(x['inspection_type'])
    dlist.append(x['license_'])
    dlist.append(x['results'])
    dlist.append(x['risk'])
    try:
        dlist.append(x['state'])
    except KeyError:
        dlist.append(' ')
    try:
        dlist.append(x['violations'])
    except:
        dlist.append(' ')
    dlist.append(x['zip'])
    data_rows.append(dlist)

In [10]:
# Create a 2nd dataframe with data_rows and column header list
df = pd.DataFrame(data_rows, columns=clist)
df.head()

,address,aka_name,city,business_name,facility_type,inspection_date,inspection_id,inspection_type,license_id,results,risk,state_id,violations,zip
0,2324 W GIDDINGS ST,PACIUGO CHICAGO,CHICAGO,PACIUGO CHICAGO,Mobile Food Dispenser,2023-01-17T00:00:00.000,2569731,Canvass,2240574,Out of Business,Risk 3 (Low),IL,,60625
1,1731-1735 N ELSTON AVE,CREATIVE SCHOLARS PRESCHOOL,CHICAGO,CREATIVE SCHOLARS PRESCHOOL,Children's Services Facility,2023-01-17T00:00:00.000,2569739,License,2216013,Pass,Risk 1 (High),IL,"53. TOILET FACILITIES: PROPERLY CONSTRUCTED, SUPPLIED, & CLEANED - Comments: OBSERVED ONE UNISEX STAFF RESTROOM MISSING A COVERED TRASH RECEPTACLE. INSTRUCTED TO PROVIDE.",60622
2,6064-6070 N NORTHWEST HWY,MYSTIC ROGUE,CHICAGO,MYSTIC ROGUE,Restaurant,2023-01-13T00:00:00.000,2569702,Canvass,2708621,Pass,Risk 1 (High),IL,"16. FOOD-CONTACT SURFACES: CLEANED & SANITIZED - Comments: OBSERVED CANOPENER UNCLEAN. CLEAN AND SANITIZE CAN OPENER AND MAINTAIN. | 47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE, PROPERLY DESIGN...",60631
3,2016 1/2 W DIVISION ST,PICANTE TAQUERIA,CHICAGO,PICANTE TAQUERIA,Restaurant,2023-01-13T00:00:00.000,2569716,Canvass,1938696,Out of Business,Risk 1 (High),IL,,60622
4,2414 W 103RD ST,SWANSON'S DELI & BEVERLY RITZ CATERING,CHICAGO,SWANSON'S BEVERLY RITZ,Restaurant,2023-01-13T00:00:00.000,2569684,Canvass,2510601,Fail,Risk 1 (High),IL,"38. INSECTS, RODENTS, & ANIMALS NOT PRESENT - Comments: OBSERVED OVER 50 MICE DROPPINGS SCATTERED ON SLICER, ROLLING CARTS, INSIDE STORAGE BINS, ON FLOOR, SHELVES AND EQUIPMENT IN BASEMENT STORAGE...",60655


In [11]:
# Change Date and some numeric data columns to appropriate data types
df['license_id'] = df['license_id'].astype(int)
df['inspection_id'] = df['inspection_id'].astype(int)
df['zip'] = df['zip'].astype(int)

# 2 step process to get just date portion and change data type
from datetime import datetime as dt
df['inspection_date'] = pd.to_datetime(df["inspection_date"]).dt.strftime('%Y-%m-%d')
df["inspection_date"] = pd.to_datetime(df["inspection_date"])


In [12]:
# Reorder columns
df = df[['inspection_id', 'business_name', 'aka_name', 'license_id', 'facility_type', 'risk', 'address', 'city',
        'state_id', 'zip', 'inspection_date', 'inspection_type', 'results', 'violations']]
df.head()

,inspection_id,business_name,aka_name,license_id,facility_type,risk,address,city,state_id,zip,inspection_date,inspection_type,results,violations
0,2569731,PACIUGO CHICAGO,PACIUGO CHICAGO,2240574,Mobile Food Dispenser,Risk 3 (Low),2324 W GIDDINGS ST,CHICAGO,IL,60625,2023-01-17,Canvass,Out of Business,
1,2569739,CREATIVE SCHOLARS PRESCHOOL,CREATIVE SCHOLARS PRESCHOOL,2216013,Children's Services Facility,Risk 1 (High),1731-1735 N ELSTON AVE,CHICAGO,IL,60622,2023-01-17,License,Pass,"53. TOILET FACILITIES: PROPERLY CONSTRUCTED, SUPPLIED, & CLEANED - Comments: OBSERVED ONE UNISEX STAFF RESTROOM MISSING A COVERED TRASH RECEPTACLE. INSTRUCTED TO PROVIDE."
2,2569702,MYSTIC ROGUE,MYSTIC ROGUE,2708621,Restaurant,Risk 1 (High),6064-6070 N NORTHWEST HWY,CHICAGO,IL,60631,2023-01-13,Canvass,Pass,"16. FOOD-CONTACT SURFACES: CLEANED & SANITIZED - Comments: OBSERVED CANOPENER UNCLEAN. CLEAN AND SANITIZE CAN OPENER AND MAINTAIN. | 47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE, PROPERLY DESIGN..."
3,2569716,PICANTE TAQUERIA,PICANTE TAQUERIA,1938696,Restaurant,Risk 1 (High),2016 1/2 W DIVISION ST,CHICAGO,IL,60622,2023-01-13,Canvass,Out of Business,
4,2569684,SWANSON'S BEVERLY RITZ,SWANSON'S DELI & BEVERLY RITZ CATERING,2510601,Restaurant,Risk 1 (High),2414 W 103RD ST,CHICAGO,IL,60655,2023-01-13,Canvass,Fail,"38. INSECTS, RODENTS, & ANIMALS NOT PRESENT - Comments: OBSERVED OVER 50 MICE DROPPINGS SCATTERED ON SLICER, ROLLING CARTS, INSIDE STORAGE BINS, ON FLOOR, SHELVES AND EQUIPMENT IN BASEMENT STORAGE..."


In [13]:
# Inspect dataframe column types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   inspection_id    1000 non-null   int64         
 1   business_name    1000 non-null   object        
 2   aka_name         1000 non-null   object        
 3   license_id       1000 non-null   int64         
 4   facility_type    1000 non-null   object        
 5   risk             1000 non-null   object        
 6   address          1000 non-null   object        
 7   city             1000 non-null   object        
 8   state_id         1000 non-null   object        
 9   zip              1000 non-null   int64         
 10  inspection_date  1000 non-null   datetime64[ns]
 11  inspection_type  1000 non-null   object        
 12  results          1000 non-null   object        
 13  violations       1000 non-null   object        
dtypes: datetime64[ns](1), int64(3), object(10

In [14]:
# Compare above 2nd dataset columns against 1st dataset columns - verify columns name, type, and order aligns
name_change_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172880 entries, 0 to 172879
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   inspection_id    172880 non-null  int64  
 1   business_name    172880 non-null  object 
 2   aka_name         170663 non-null  object 
 3   license_id       172872 non-null  float64
 4   facility_type    168288 non-null  object 
 5   risk             172819 non-null  object 
 6   address          172880 non-null  object 
 7   city             172787 non-null  object 
 8   state_id         172858 non-null  object 
 9   zip              172840 non-null  float64
 10  inspection_date  172880 non-null  object 
 11  inspection_type  172879 non-null  object 
 12  results          172880 non-null  object 
 13  violations       126458 non-null  object 
dtypes: float64(2), int64(1), object(11)
memory usage: 18.5+ MB


In [15]:
# Bring the two datasets as one
food_inspection_df = pd.concat([name_change_df,df])
food_inspection_df.head(50)

,inspection_id,business_name,aka_name,license_id,facility_type,risk,address,city,state_id,zip,inspection_date,inspection_type,results,violations
0,67757,DUNKIN DONUTS/BASKIN-ROBBINS,DUNKIN DONUTS/BASKIN-ROBBINS,1380279.0,Restaurant,Risk 2 (Medium),100 W RANDOLPH ST,CHICAGO,IL,60601.0,01/04/2010,Tag Removal,Pass,NaN
1,52234,Cafe 608,Cafe 608,2013328.0,Restaurant,Risk 1 (High),608 W BARRY AVE,CHICAGO,IL,60657.0,01/04/2010,License Re-Inspection,Pass,NaN
2,70269,mr.daniel's,mr.daniel's,1899292.0,Restaurant,Risk 1 (High),5645 W BELMONT AVE,CHICAGO,IL,60634.0,01/04/2010,License Re-Inspection,Pass,NaN
3,67733,WOLCOTT'S,TROQUET,1992040.0,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613.0,01/04/2010,License Re-Inspection,Pass,NaN
4,104236,TEMPO CAFE,TEMPO CAFE,80916.0,Restaurant,Risk 1 (High),6 E CHESTNUT ST,CHICAGO,IL,60611.0,01/04/2010,Canvass,Fail,"18. NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS PROTECTED/RODENT PROOFED, A WRITTEN LOG SHALL BE MAINTAINED AVAILABLE TO THE INSPECTORS - Comments: All necessary control measures shall be used ..."
5,67732,WOLCOTT'S,TROQUET,1992039.0,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613.0,01/04/2010,License Re-Inspection,Pass,NaN
6,67738,MICHAEL'S ON MAIN CAFE,MICHAEL'S ON MAIN CAFE,2008948.0,Restaurant,Risk 1 (High),8750 W BRYN WAWR AVE,CHICAGO,IL,60631.0,01/04/2010,License,Fail,"18. NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS PROTECTED/RODENT PROOFED, A WRITTEN LOG SHALL BE MAINTAINED AVAILABLE TO THE INSPECTORS - Comments: must provide pest control log book. | 32. FOO..."
7,67760,MC INTOSH'S,MC INTOSH'S,1648623.0,NaN,Risk 1 (High),748 W 61ST ST,CHICAGO,IL,60621.0,01/05/2010,Out of Business,Fail,NaN
8,78240,OBAMA FOOD MART,NaN,2004238.0,Grocery Store,Risk 3 (Low),1800 E 79TH ST,CHICAGO,IL,60649.0,01/05/2010,License,Fail,NaN
9,74258,MAKIA FOOD,MAKIA FOOD,2009488.0,NaN,Risk 3 (Low),2458 S CALIFORNIA AVE,CHICAGO,IL,60608.0,01/05/2010,License Re-Inspection,Pass,NaN


In [16]:
# Inspect final dataframe raw types
food_inspection_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173880 entries, 0 to 999
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   inspection_id    173880 non-null  int64  
 1   business_name    173880 non-null  object 
 2   aka_name         171663 non-null  object 
 3   license_id       173872 non-null  float64
 4   facility_type    169288 non-null  object 
 5   risk             173819 non-null  object 
 6   address          173880 non-null  object 
 7   city             173787 non-null  object 
 8   state_id         173858 non-null  object 
 9   zip              173840 non-null  float64
 10  inspection_date  173880 non-null  object 
 11  inspection_type  173879 non-null  object 
 12  results          173880 non-null  object 
 13  violations       127458 non-null  object 
dtypes: float64(2), int64(1), object(11)
memory usage: 19.9+ MB


In [17]:
# Cleanup the Risk Column - split into 3 as part 1 of cleanup
food_inspection_df[['risk_delete','risk_rank','risk_category']] = food_inspection_df['risk'].str.split(n=-1, expand=True)
food_inspection_df.head()

,inspection_id,business_name,aka_name,license_id,facility_type,risk,address,city,state_id,zip,inspection_date,inspection_type,results,violations,risk_delete,risk_rank,risk_category
0,67757,DUNKIN DONUTS/BASKIN-ROBBINS,DUNKIN DONUTS/BASKIN-ROBBINS,1380279.0,Restaurant,Risk 2 (Medium),100 W RANDOLPH ST,CHICAGO,IL,60601.0,01/04/2010,Tag Removal,Pass,NaN,Risk,2,(Medium)
1,52234,Cafe 608,Cafe 608,2013328.0,Restaurant,Risk 1 (High),608 W BARRY AVE,CHICAGO,IL,60657.0,01/04/2010,License Re-Inspection,Pass,NaN,Risk,1,(High)
2,70269,mr.daniel's,mr.daniel's,1899292.0,Restaurant,Risk 1 (High),5645 W BELMONT AVE,CHICAGO,IL,60634.0,01/04/2010,License Re-Inspection,Pass,NaN,Risk,1,(High)
3,67733,WOLCOTT'S,TROQUET,1992040.0,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613.0,01/04/2010,License Re-Inspection,Pass,NaN,Risk,1,(High)
4,104236,TEMPO CAFE,TEMPO CAFE,80916.0,Restaurant,Risk 1 (High),6 E CHESTNUT ST,CHICAGO,IL,60611.0,01/04/2010,Canvass,Fail,"18. NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS PROTECTED/RODENT PROOFED, A WRITTEN LOG SHALL BE MAINTAINED AVAILABLE TO THE INSPECTORS - Comments: All necessary control measures shall be used ...",Risk,1,(High)


In [18]:
# Get rid of parenthesis () in Risk_Category column - part 2 of cleanup
food_inspection_df['risk_category'] = food_inspection_df['risk_category'].str.replace('(', '', regex=False)
food_inspection_df['risk_category'] = food_inspection_df['risk_category'].str.replace(')', '', regex=False)
food_inspection_df.head()

,inspection_id,business_name,aka_name,license_id,facility_type,risk,address,city,state_id,zip,inspection_date,inspection_type,results,violations,risk_delete,risk_rank,risk_category
0,67757,DUNKIN DONUTS/BASKIN-ROBBINS,DUNKIN DONUTS/BASKIN-ROBBINS,1380279.0,Restaurant,Risk 2 (Medium),100 W RANDOLPH ST,CHICAGO,IL,60601.0,01/04/2010,Tag Removal,Pass,NaN,Risk,2,Medium
1,52234,Cafe 608,Cafe 608,2013328.0,Restaurant,Risk 1 (High),608 W BARRY AVE,CHICAGO,IL,60657.0,01/04/2010,License Re-Inspection,Pass,NaN,Risk,1,High
2,70269,mr.daniel's,mr.daniel's,1899292.0,Restaurant,Risk 1 (High),5645 W BELMONT AVE,CHICAGO,IL,60634.0,01/04/2010,License Re-Inspection,Pass,NaN,Risk,1,High
3,67733,WOLCOTT'S,TROQUET,1992040.0,Restaurant,Risk 1 (High),1834 W MONTROSE AVE,CHICAGO,IL,60613.0,01/04/2010,License Re-Inspection,Pass,NaN,Risk,1,High
4,104236,TEMPO CAFE,TEMPO CAFE,80916.0,Restaurant,Risk 1 (High),6 E CHESTNUT ST,CHICAGO,IL,60611.0,01/04/2010,Canvass,Fail,"18. NO EVIDENCE OF RODENT OR INSECT OUTER OPENINGS PROTECTED/RODENT PROOFED, A WRITTEN LOG SHALL BE MAINTAINED AVAILABLE TO THE INSPECTORS - Comments: All necessary control measures shall be used ...",Risk,1,High


In [20]:
# Drop unnecessary columns to finalize Risk cleanup - part 3
food_inspection_df = food_inspection_df.drop('risk_delete', axis=1)
food_inspection_df = food_inspection_df.drop('risk', axis=1)
food_inspection_df = food_inspection_df.drop('violations', axis=1)


In [21]:
food_inspection_df.head()

,inspection_id,business_name,aka_name,license_id,facility_type,address,city,state_id,zip,inspection_date,inspection_type,results,risk_rank,risk_category
0,67757,DUNKIN DONUTS/BASKIN-ROBBINS,DUNKIN DONUTS/BASKIN-ROBBINS,1380279.0,Restaurant,100 W RANDOLPH ST,CHICAGO,IL,60601.0,01/04/2010,Tag Removal,Pass,2,Medium
1,52234,Cafe 608,Cafe 608,2013328.0,Restaurant,608 W BARRY AVE,CHICAGO,IL,60657.0,01/04/2010,License Re-Inspection,Pass,1,High
2,70269,mr.daniel's,mr.daniel's,1899292.0,Restaurant,5645 W BELMONT AVE,CHICAGO,IL,60634.0,01/04/2010,License Re-Inspection,Pass,1,High
3,67733,WOLCOTT'S,TROQUET,1992040.0,Restaurant,1834 W MONTROSE AVE,CHICAGO,IL,60613.0,01/04/2010,License Re-Inspection,Pass,1,High
4,104236,TEMPO CAFE,TEMPO CAFE,80916.0,Restaurant,6 E CHESTNUT ST,CHICAGO,IL,60611.0,01/04/2010,Canvass,Fail,1,High


In [22]:
food_inspection_df.count()

inspection_id      173880
business_name      173880
aka_name           171663
license_id         173872
facility_type      169288
address            173880
city               173787
state_id           173858
zip                173840
inspection_date    173880
inspection_type    173879
results            173880
risk_rank          173799
risk_category      173799
dtype: int64

In [23]:
# Data Cleanup - Remove inspections from businesss no longer in business
food_inspection_df_clean= food_inspection_df[food_inspection_df.results != 'Out of Business']

In [24]:
food_inspection_df_clean.count()

inspection_id      158554
business_name      158554
aka_name           156868
license_id         158547
facility_type      157838
address            158554
city               158478
state_id           158534
zip                158528
inspection_date    158554
inspection_type    158553
results            158554
risk_rank          158479
risk_category      158479
dtype: int64

In [26]:
import numpy as np

In [31]:
# Cleanup the numeric data & types
food_inspection_df_clean['risk_rank'] = food_inspection_df_clean['risk_rank'].replace(np.nan, 0)
food_inspection_df_clean['zip'] = food_inspection_df_clean['zip'].replace(np.nan, 0)
food_inspection_df_clean['risk_rank'] = food_inspection_df_clean['risk_rank'].astype(int)
food_inspection_df_clean['zip'] = food_inspection_df_clean['zip'].astype(int)
food_inspection_df_clean['license_id'] = food_inspection_df_clean['license_id'].replace(np.nan, 0)
food_inspection_df_clean['license_id'] = food_inspection_df_clean['license_id'].astype(int)

/var/folders/r1/w1303vn5085bcrsmp_83wz2r0000gn/T/ipykernel_20974/4094350822.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_inspection_df_clean['risk_rank'] = food_inspection_df_clean['risk_rank'].replace(np.nan, 0)
/var/folders/r1/w1303vn5085bcrsmp_83wz2r0000gn/T/ipykernel_20974/4094350822.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  food_inspection_df_clean['zip'] = food_inspection_df_clean['zip'].replace(np.nan, 0)
/var/folders/r1/w1303vn5085bcrsmp_83wz2r0000gn/T/ipykernel_20974/409435082

In [32]:
food_inspection_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158554 entries, 0 to 999
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   inspection_id    158554 non-null  int64 
 1   business_name    158554 non-null  object
 2   aka_name         156868 non-null  object
 3   license_id       158554 non-null  int64 
 4   facility_type    157838 non-null  object
 5   address          158554 non-null  object
 6   city             158478 non-null  object
 7   state_id         158534 non-null  object
 8   zip              158554 non-null  int64 
 9   inspection_date  158554 non-null  object
 10  inspection_type  158553 non-null  object
 11  results          158554 non-null  object
 12  risk_rank        158554 non-null  int64 
 13  risk_category    158479 non-null  object
dtypes: int64(4), object(10)
memory usage: 18.1+ MB


In [33]:
food_inspection_df_clean.head()

,inspection_id,business_name,aka_name,license_id,facility_type,address,city,state_id,zip,inspection_date,inspection_type,results,risk_rank,risk_category
0,67757,DUNKIN DONUTS/BASKIN-ROBBINS,DUNKIN DONUTS/BASKIN-ROBBINS,1380279,Restaurant,100 W RANDOLPH ST,CHICAGO,IL,60601,01/04/2010,Tag Removal,Pass,2,Medium
1,52234,Cafe 608,Cafe 608,2013328,Restaurant,608 W BARRY AVE,CHICAGO,IL,60657,01/04/2010,License Re-Inspection,Pass,1,High
2,70269,mr.daniel's,mr.daniel's,1899292,Restaurant,5645 W BELMONT AVE,CHICAGO,IL,60634,01/04/2010,License Re-Inspection,Pass,1,High
3,67733,WOLCOTT'S,TROQUET,1992040,Restaurant,1834 W MONTROSE AVE,CHICAGO,IL,60613,01/04/2010,License Re-Inspection,Pass,1,High
4,104236,TEMPO CAFE,TEMPO CAFE,80916,Restaurant,6 E CHESTNUT ST,CHICAGO,IL,60611,01/04/2010,Canvass,Fail,1,High


In [34]:
# Find duplicate IDs
print(food_inspection_df_clean.count())

food_inspection_df_clean['inspection_id'].duplicated()
food_inspection_df_clean['inspection_id'].duplicated().sum()

inspection_id      158554
business_name      158554
aka_name           156868
license_id         158554
facility_type      157838
address            158554
city               158478
state_id           158534
zip                158554
inspection_date    158554
inspection_type    158553
results            158554
risk_rank          158554
risk_category      158479
dtype: int64


0

In [35]:
# Remove Duplicate Rows based on 'inspection_id'
food_inspection_df_clean = food_inspection_df_clean.drop_duplicates(subset='inspection_id')
food_inspection_df_clean.count()

inspection_id      158554
business_name      158554
aka_name           156868
license_id         158554
facility_type      157838
address            158554
city               158478
state_id           158534
zip                158554
inspection_date    158554
inspection_type    158553
results            158554
risk_rank          158554
risk_category      158479
dtype: int64

In [36]:
food_inspection_df_clean.head()

,inspection_id,business_name,aka_name,license_id,facility_type,address,city,state_id,zip,inspection_date,inspection_type,results,risk_rank,risk_category
0,67757,DUNKIN DONUTS/BASKIN-ROBBINS,DUNKIN DONUTS/BASKIN-ROBBINS,1380279,Restaurant,100 W RANDOLPH ST,CHICAGO,IL,60601,01/04/2010,Tag Removal,Pass,2,Medium
1,52234,Cafe 608,Cafe 608,2013328,Restaurant,608 W BARRY AVE,CHICAGO,IL,60657,01/04/2010,License Re-Inspection,Pass,1,High
2,70269,mr.daniel's,mr.daniel's,1899292,Restaurant,5645 W BELMONT AVE,CHICAGO,IL,60634,01/04/2010,License Re-Inspection,Pass,1,High
3,67733,WOLCOTT'S,TROQUET,1992040,Restaurant,1834 W MONTROSE AVE,CHICAGO,IL,60613,01/04/2010,License Re-Inspection,Pass,1,High
4,104236,TEMPO CAFE,TEMPO CAFE,80916,Restaurant,6 E CHESTNUT ST,CHICAGO,IL,60611,01/04/2010,Canvass,Fail,1,High


In [39]:
# Form connection string for accessing PostgreSQL database
conn_string = f"postgresql://{credentials}@localhost:5432/food_inspection_db"  

In [40]:
# Create sqlalchemy engine using connection string
engine = create_engine(conn_string)
conn = engine.connect()

In [41]:
# Create a dictionary for bulk insert
bulk_list = food_inspection_df_clean.to_dict(orient='records')

In [42]:
# Connect to our Inspections table
table_name = 'inspection'
metadata = sqlalchemy.schema.MetaData(bind=engine)
table = sqlalchemy.Table(table_name, metadata, autoload=True)

In [43]:
# session = Session(bind=engine)
Session = sessionmaker(bind=engine)

In [44]:
# Bulk Insert the dataframe into the database table

with Session() as session:
    food_inspection_df_clean.to_sql(table_name, con=conn, if_exists='append', index=False)

In [45]:
# Commit the changes
session.commit()

# close session
session.close()

# close connection
conn.close()